In [1]:
from sqlalchemy import create_engine, Column, String, Integer, Boolean, MetaData, Table, case, between, ForeignKey
from sqlalchemy.orm import mapper, create_session, relationship
from sqlalchemy.ext.declarative import declarative_base
import sqlalchemy.types as types
from sqlalchemy.sql import select, and_, or_, not_, func
from time import strptime
import csv
import re
import os
import glob
import numpy as np
import string
import pandas as pd
import seawater as sw # for sw.dpth(p,lat) 
import warnings
import datetime as dt
from dateutil.parser import parse as dutparse
from dataProcess import forceNumeric, forceInt, fmtVarName, data2Tbl, adFunONC, isNum

In [2]:
basepath='/ocean/eolson/MEOPAR/obs/'
basedir=basepath + 'ECBuoy/'
dbname='ECBuoy'

fout=open(basedir+'createDBfromECBuoy_log.txt','w')

if os.path.isfile(basedir + dbname + '.sqlite'):
    os.remove(basedir + dbname + '.sqlite')
engine = create_engine('sqlite:///' + basedir + dbname + '.sqlite')
Base=declarative_base()

# define Table Classes

class FBuoyTBL(Base):
    __table__=Table('FBuoyTBL', Base.metadata,
                    Column('ID', Integer, primary_key=True),
                    Column('Time_UTC', String),
                    Column('DecDay', forceNumeric, index=True),
                    Column('Tem', forceNumeric),
                    Column('Turb_NTU', forceNumeric),
                    Column('SR', forceNumeric),
                    Column('O2', forceNumeric),
                    Column('sourceFile', String),
                    )
    
Base.metadata.create_all(engine)
session = create_session(bind = engine, autocommit = False, autoflush = True)

In [3]:
def adFunECB(row):
    idate=dutparse(row['dates'])#+dt.timedelta(hours=8) ## column named datatimestamp is in UTC
    row['Time_UTC']=idate.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3]+'Z'
    row['DecDay']=(idate.date()-dt.date(1900,1,1)).days+float(idate.hour)/24.0+float(idate.minute)/1440.0+float(idate.second)/86400.0 
    del row['dates']
    return row

fpath='/ocean/eolson/MEOPAR/obs/ECBuoy/exportbuoy.txt'
header=('dates','turb','temp','sal','O2')
trDict={'dates':'dates','turb':'Turb_NTU','temp':'Tem','sal':'SR','O2':'O2'}
adlValDict={'sourceFile':'exportbuoy.txt'}
reqDta=('Turb_NTU',)
data2Tbl(fpath,session,FBuoyTBL,1,',',header,trDict,adlValDict,reqDta,adFunECB)

fpath2='/ocean/eolson/MEOPAR/obs/ECBuoy/BuoyWQData2016-10-31to20170721NoOverlap.csv'
header2=('dates','temp','specCond','ph','turb','O2Per','O2mlg')
trDict2={'dates':'dates','turb':'Turb_NTU','temp':'Tem','O2mlg':'O2'}
adlValDict2={'sourceFile':'BuoyWQData2016-10-31to20170721NoOverlap.csv'}
data2Tbl(fpath2,session,FBuoyTBL,1,',',header2,trDict2,adlValDict2,reqDta,adFunECB)

def adFunECBPac(row):
    idate=dutparse(row['dates'])+dt.timedelta(hours=8) ## these times are UTC-8
    row['Time_UTC']=idate.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3]+'Z'
    row['DecDay']=(idate.date()-dt.date(1900,1,1)).days+float(idate.hour)/24.0+float(idate.minute)/1440.0+float(idate.second)/86400.0 
    del row['dates']
    return row

fpath3='/ocean/eolson/MEOPAR/obs/ECBuoy/FRBWQ_20170720to20190617NoOverlap.csv'
header3=('dates','temp','specCond','ph','turb','O2mlg')
trDict3={'dates':'dates','turb':'Turb_NTU','temp':'Tem','O2mlg':'O2'}
adlValDict3={'sourceFile':'FRBWQ_20170720to20190617NoOverlap.csv'}
data2Tbl(fpath3,session,FBuoyTBL,3,',',header3,trDict3,adlValDict3,reqDta,adFunECBPac)


In [4]:
print([row for row in session.query(FBuoyTBL.ID,FBuoyTBL.Time_UTC,FBuoyTBL.DecDay,FBuoyTBL.Turb_NTU).limit(5)])

[(1, '2009-01-01T00:10:21.000Z', 39812.0071875, 6.7), (2, '2009-01-01T01:10:21.000Z', 39812.048854166664, 8.2), (3, '2009-01-01T02:10:21.000Z', 39812.090520833335, 14.8), (4, '2009-01-01T04:10:21.000Z', 39812.173854166664, 5.3), (5, '2009-01-01T05:10:21.000Z', 39812.215520833335, 5.1)]


In [5]:
session.close()
engine.dispose()

In [6]:
# add to ONC db

In [7]:
basedir='/ocean/eolson/MEOPAR/obs/ONC/'
dbname='ONC'

engine = create_engine('sqlite:///' + basedir + dbname + '.sqlite', echo = False)
Base=declarative_base(engine)

connection=engine.connect()
if engine.dialect.has_table(connection,'FBuoyTBL'):
    # delete existing ModelGridTBL
    connection.execute('DROP TABLE FBuoyTBL')
connection.close()

# reflect existing tables
class HBDBTurbTBL(Base):
    __tablename__= 'HBDBTurbTBL'
    __table_args__= {'autoload':True}
class HBDBSalTBL(Base):
    __tablename__= 'HBDBSalTBL'
    __table_args__= {'autoload':True}
class TDPTurbTBL(Base):
    __tablename__= 'TDPTurbTBL'
    __table_args__= {'autoload':True}
class TDPSalTBL(Base):
    __tablename__= 'TDPSalTBL'
    __table_args__= {'autoload':True}
class TSBTurbTBL(Base):
    __tablename__= 'TSBTurbTBL'
    __table_args__= {'autoload':True}
class TSBSalTBL(Base):
    __tablename__= 'TSBSalTBL'
    __table_args__= {'autoload':True}

# create new Table
class FBuoyTBL(Base):
    __table__=Table('FBuoyTBL', Base.metadata,
                    Column('ID', Integer, primary_key=True),
                    Column('Time_UTC', String),
                    Column('DecDay', forceNumeric, index=True),
                    Column('Tem', forceNumeric),
                    Column('Turb_NTU', forceNumeric),
                    Column('SR', forceNumeric),
                    Column('O2', forceNumeric),
                    Column('sourceFile', String))
        
Base.metadata.create_all(engine)
session = create_session(bind = engine, autocommit = False, autoflush = True)

fpath='/ocean/eolson/MEOPAR/obs/ECBuoy/exportbuoy.txt'
header=('dates','turb','temp','sal','O2')
trDict={'dates':'dates','turb':'Turb_NTU','temp':'Tem','sal':'SR','O2':'O2'}
adlValDict={'sourceFile':'exportbuoy.txt'}
reqDta=('Turb_NTU',)

def adFunECB(row):
    idate=dutparse(row['dates'])#+dt.timedelta(hours=8)
    row['Time_UTC']=idate.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3]+'Z'
    row['DecDay']=(idate.date()-dt.date(1900,1,1)).days+float(idate.hour)/24.0+float(idate.minute)/1440.0+float(idate.second)/86400.0 
    del row['dates']
    return row

data2Tbl(fpath,session,FBuoyTBL,1,',',header,trDict,adlValDict,reqDta,adFunECB)

fpath2='/ocean/eolson/MEOPAR/obs/ECBuoy/BuoyWQData2016-10-31toPresentToCSV.csv'
header2=('dates','temp','specCond','ph','turb','O2Per','O2mlg')
trDict2={'dates':'dates','turb':'Turb_NTU','temp':'Tem','O2mlg':'O2'}
adlValDict2={'sourceFile':'BuoyWQData2016-10-31toPresentToCSV.csv'}

data2Tbl(fpath2,session,FBuoyTBL,1,',',header2,trDict2,adlValDict2,reqDta,adFunECB)

def adFunECBPac(row):
    idate=dutparse(row['dates'])+dt.timedelta(hours=8) ## these times are UTC-8
    row['Time_UTC']=idate.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3]+'Z'
    row['DecDay']=(idate.date()-dt.date(1900,1,1)).days+float(idate.hour)/24.0+float(idate.minute)/1440.0+float(idate.second)/86400.0 
    del row['dates']
    return row

fpath3='/ocean/eolson/MEOPAR/obs/ECBuoy/FRBWQ_20170720to20190617NoOverlap.csv'
header3=('dates','temp','specCond','ph','turb','O2mlg')
trDict3={'dates':'dates','turb':'Turb_NTU','temp':'Tem','O2mlg':'O2'}
adlValDict3={'sourceFile':'FRBWQ_20170720to20190617NoOverlap.csv'}
data2Tbl(fpath3,session,FBuoyTBL,3,',',header3,trDict3,adlValDict3,reqDta,adFunECBPac)



print([row for row in session.query(FBuoyTBL.ID,FBuoyTBL.Time_UTC,FBuoyTBL.DecDay,FBuoyTBL.Turb_NTU).limit(5)])
session.close()
engine.dispose()

NoSuchTableError: HBDBTurbTBL